In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Toy example 1 - Single eligible group, Single ineligible group

In [2]:
def _extract_solution_discount(m):
    """
    Get solution from optimization model
    """
    users_in = [v.x for v in m.getVars() if v.VarName.find("ineligible_val") != -1]
    users_el = [v.x for v in m.getVars() if v.VarName.find("eligible_allocation") != -1]
    edge_flows = [v.x for v in m.getVars() if v.VarName.find("edge_flows") != -1]
    
    users_in = np.reshape(users_in, (2, T))
    users_el = np.reshape(users_el, (2, T))
    edge_flows = np.reshape(edge_flows, (2, T))

    solution = {
        "users_in": users_in,
        "users_el": users_el,
        "edge_flows": edge_flows
    }
    
    return solution

In [44]:
def avg_travel_times_1(solution_set, theta):
    
    users_in = solution_set["users_in"]
    users_el = solution_set["users_el"]
    edge_flows = solution_set["edge_flows"]
    
    avg_travel_time_in = sum( users_in[j, t] * (theta[j,0] + edge_flows[j, t] * theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_el = sum( users_el[j, t] * (theta[j,0] + edge_flows[j, t] * theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_all = sum( edge_flows[j, t] * (theta[j,0] + edge_flows[j, t] * theta[j,1]) for j in range(2) for t in range(T)) / (T*2)
    
    avg_travel_times = {
        "users_in": avg_travel_time_in,
        "users_el": avg_travel_time_el,
        "users_all": avg_travel_time_all
    }
    
    return avg_travel_times

# def percentage_users()


In [45]:
def m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_d1 = gp.Model()

    users_in = m_d1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    users_el = m_d1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
    edge_flows = m_d1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_d1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_d1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_d1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_d1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_d1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_d1.setObjective(sum( a_1*edge_flows[0, t] + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*edge_flows[1, t] + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
                        + (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible 
        for t in range(T)), GRB.MINIMIZE)
    
    m_d1.update()

    m_d1.optimize()
    
    solution_set_d1 = _extract_solution_discount(m_d1)

    return solution_set_d1

In [58]:
def m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_b1 = gp.Model()

    users_in = m_b1.addVars(3, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    users_el = m_b1.addVars(3, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
    edge_flows = m_b1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_b1.addConstrs((edge_flows[0, t] == users_in[0, t] + users_el[0, t] + users_in[1, t] + users_el[1, t] for t in range(T)))
    m_b1.addConstrs((edge_flows[1, t] == users_in[2, t] + users_el[2, t] for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_b1.addConstrs((users_in[j, t] >= 0 for j in range(3) for t in range(T)))
    m_b1.addConstrs((users_el[j, t] >= 0 for j in range(3) for t in range(T)))
    m_b1.addConstrs((users_el[0, t] == 0 for t in range(T)))
    
    # Budget constraint satisfaction:
    if toll != 0:
        m_b1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= budget))
    m_b1.addConstrs((users_in[0, t] == 0 for t in range(T)))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_b1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_b1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_b1.setObjective(sum( a_1 * (edge_flows[0, t]) + b_1 * (edge_flows[0, t]**2)/2  
                        + a_2 * (edge_flows[1, t]) + b_2 * (edge_flows[1, t]**2)/2 
                        + toll * users_in[1,t]/vot_ineligible
                        + toll * users_el[1,t]/vot_eligible
        for t in range(T)), GRB.MINIMIZE)
    
    m_b1.update()

    m_b1.optimize()
    
    solution_set_b1 = _extract_solution_discount(m_b1)

    return solution_set_b1

In [59]:
def m_bpure_1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_bpure_1 = gp.Model()

    users_in = m_bpure_1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    users_el = m_bpure_1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
    edge_flows = m_bpure_1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_bpure_1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_bpure_1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_bpure_1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
    # Budget constraint satisfaction:
    if toll != 0:
        m_bpure_1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= budget))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_bpure_1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_bpure_1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_bpure_1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
        for t in range(T)), GRB.MINIMIZE)
    
    m_bpure_1.update()

    m_bpure_1.optimize()
    
    solution_set_bpure_1 = _extract_solution_discount(m_bpure_1)

    return solution_set_bpure_1

In [60]:
time_1 = time.time()

# Affine lane functions:
a_1 = 1.0
b_1 = 2.0
a_2 = 1.0
b_2 = 2.0

# a_1 = 0.5
# b_1 = 0.5
# a_2 = 0.5
# b_2 = 0.5

# a_1 = 4.0
# b_1 = 2.0
# a_2 = 5.0
# b_2 = 1.5

theta = np.array([[a_1, b_1], [a_2, b_2]])
T = 1
# toll = 1.0
# budget = 0.25
toll_array = (np.arange(20) + 1)/20
budget_array = (np.arange(20) + 1)*T/20

vot_ineligible = 1
vot_eligible = 1

solution_set_array_d1 = {}
solution_set_array_bpure_1 = {}
solution_set_array_b1 = {}

# Return solution:

for toll_idx, toll in enumerate(toll_array):
    
    solution_set_array_at_toll_d1 = {}
    solution_set_array_at_toll_bpure_1 = {}
    solution_set_array_at_toll_b1 = {}
    
    for budget_idx, budget in enumerate(budget_array):
        
        if budget > toll * T:
            continue
        
        discount_ratio = budget / (toll * T)
        
        print()
        print("Toll:", toll)
        print("Budget:", budget)
        print("Discount Ratio:", discount_ratio)
        print()
        
        solution_set_d1 = m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio)
        solution_set_bpure_1 = m_bpure_1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget)
        solution_set_b1 = m_bpure_1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget)
        
        solution_set_array_at_toll_d1[budget] = solution_set_d1
        solution_set_array_at_toll_bpure_1[budget] = solution_set_bpure_1
        solution_set_array_at_toll_b1[budget] = solution_set_b1
    
    solution_set_array_d1[toll] = solution_set_array_at_toll_d1
    solution_set_array_bpure_1[toll] = solution_set_array_at_toll_bpure_1
    solution_set_array_b1[toll] = solution_set_array_at_toll_b1

time_2 = time.time()
print("Time: ", time_2 - time_1)


Toll: 0.05
Budget: 0.05
Discount Ratio: 1.0

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x36155456
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual

 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.17174106e+07 -2.24398017e+08  9.13e+03 3.57e+03  1.19e+07     0s
   1   7.06460972e+01 -4.15133785e+06  8.87e+00 3.51e+00  5.06e+05     0s
   2   4.16571761e+00 -5.38223025e+03  4.50e-06 3.51e-06  6.73e+02     0s
   3   4.16471866e+00 -3.88569423e+00  2.23e-09 1.74e-09  1.01e+00     0s
   4   4.06902543e+00  3.91436405e+00  1.82e-11 1.27e-11  1.93e-02     0s
   5   4.05813084e+00  4.04410163e+00  1.82e-12 1.82e-12  1.75e-03     0s
   6   4.04876057e+00  4.04870627e+00  0.00e+00 1.39e-17  6.78e-06     0s
   7   4.04874999e+00  4.04874996e+00  0.00e+00 4.16e-17  6.78e-09     0s

Barrier solved model in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective 4.04874999e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.


Barrier solved model in 10 iterations and 0.14 seconds (0.00 work units)
Optimal objective 3.99999998e+00


Toll: 0.15
Budget: 0.05
Discount Ratio: 0.33333333333333337

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x1f1c841d
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)

   4   4.08545264e+00  3.90985762e+00  2.73e-11 2.00e-11  2.19e-02     0s
   5   4.04723241e+00  4.04090699e+00  1.82e-12 1.82e-12  7.91e-04     0s
   6   4.04718762e+00  4.04718118e+00  0.00e+00 2.50e-16  8.04e-07     0s
   7   4.04718747e+00  4.04718747e+00  0.00e+00 2.78e-17  8.04e-10     0s

Barrier solved model in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective 4.04718747e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0xba3f8940
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e-01, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e-01, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Pres

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x7a63e0b1
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.58287330e+07 -2.05133695e+08  2.12e+04 3.23e+03  1.14e+07     0s
   1   5.69809097e+01 -3.73504208e+06  1.99e+01 3.04e+00  4.56e+05     0s
   2   4.175004

...

   4   4.40464860e+00  4.06895813e+00  5.09e-11 3.27e-11  4.20e-02     0s
   5   4.27637216e+00  4.24599785e+00  3.64e-12 3.64e-12  3.80e-03     0s
   6   4.27501205e+00  4.27450442e+00  1.82e-12 0.00e+00  6.35e-05     0s
   7   4.27500004e+00  4.27499953e+00  1.82e-12 0.00e+00  6.32e-08     0s
   8   4.27499998e+00  4.27500001e+00  0.00e+00 4.44e-16  6.33e-11     0s

Barrier solved model in 8 iterations and 0.03 seconds (0.00 work units)
Optimal objective 4.27499998e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x8e4c9f64
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [6e-

 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.80161546e+07 -1.38091730e+08  6.00e+03 2.00e+03  9.51e+06     0s
   1   3.20752215e+01 -2.27333609e+06  5.37e+00 1.89e+00  2.78e+05     0s
   2   4.53182837e+00 -3.70178992e+03  2.84e-06 1.89e-06  4.63e+02     0s
   3   4.53142637e+00 -3.48569483e-01  9.02e-10 6.00e-10  6.10e-01     0s
   4   4.30769223e+00  3.90944782e+00  5.46e-11 3.82e-11  4.98e-02     0s
   5   4.13588843e+00  4.09408548e+00  5.46e-12 3.64e-12  5.23e-03     0s
   6   4.12665617e+00  4.12312436e+00  1.82e-12 1.82e-12  4.41e-04     0s
   7   4.12522405e+00  4.12475482e+00  0.00e+00 2.22e-16  5.87e-05     0s
   8   4.12503168e+00  4.12496582e+00  0.00e+00 0.00e+00  8.23e-06     0s
   9   4.12500444e+00  4.12499517e+00  0.00e+00 1.11e-16  1.16e-06     0s
  10   4.12500066e+00  4.12499928e+00  0.00e+00 1.82e-12  

   6   4.14755031e+00  4.14489322e+00  0.00e+00 0.00e+00  3.32e-04     0s
   7   4.14719000e+00  4.14705958e+00  1.82e-12 5.55e-17  1.63e-05     0s
   8   4.14718750e+00  4.14718738e+00  1.82e-12 1.74e-12  1.60e-08     0s
   9   4.14718750e+00  4.14718747e+00  0.00e+00 1.82e-12  1.60e-11     0s

Barrier solved model in 9 iterations and 0.15 seconds (0.00 work units)
Optimal objective 4.14718750e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x6eed641e
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [8e-01, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Pres

 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.80161860e+07 -1.38092016e+08  6.00e+03 2.00e+03  9.51e+06     0s
   1   3.12069011e+01 -2.27133321e+06  5.15e+00 1.86e+00  2.77e+05     0s
   2   4.50603625e+00 -3.80926396e+03  2.79e-06 1.86e-06  4.77e+02     0s
   3   4.50541073e+00 -1.14321095e+00  1.34e-09 8.95e-10  7.06e-01     0s
   4   4.27080181e+00  3.78515676e+00  8.55e-11 5.82e-11  6.07e-02     0s
   5   4.02291444e+00  4.01373228e+00  1.82e-12 5.55e-17  1.15e-03     0s
   6   4.02001768e+00  4.01999369e+00  0.00e+00 1.82e-12  3.00e-06     0s
   7   4.02000004e+00  4.01999998e+00  0.00e+00 0.00e+00  3.00e-09     0s

Barrier solved model in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective 4.02000004e+00


Toll: 1.0
Budget: 0.95
Discount Ratio: 0.95

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac


Barrier solved model in 12 iterations and 0.04 seconds (0.00 work units)
Optimal objective 4.00000000e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x95a2cc2a
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                R

In [62]:
# Average travel times, discount minus budget:

avg_travel_time_discount_minus_budget = np.zeros((toll_array.shape[0], budget_array.shape[0], 6))
# 0: in
# 1: el
# 2: all
# 3: in (pure)
# 4: el (pure)
# 5: all (pure)

for toll_idx, toll in enumerate(toll_array):    
    for budget_idx, budget in enumerate(budget_array):        
        if budget > toll * T:
            continue
            
#         print("toll:", toll)
#         print("budget:", budget)
#         print()

#         diff_in = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_in"] \
#             - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_in"]
#         diff_el = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_el"] \
#             - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_el"]
#         diff_all = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_all"] \
#             - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_all"]

        diff_in = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_in"] \
            - avg_travel_times_1(solution_set_array_b1[toll][budget], theta)["users_in"]
        diff_el = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_el"] \
            - avg_travel_times_1(solution_set_array_b1[toll][budget], theta)["users_el"]
        diff_all = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_all"] \
            - avg_travel_times_1(solution_set_array_b1[toll][budget], theta)["users_all"]
        
        diff_in_pure = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_in"] \
            - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_in"]
        diff_el_pure = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_el"] \
            - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_el"]
        diff_all_pure = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_all"] \
            - avg_travel_times_1(solution_set_array_bpure_1[toll][budget], theta)["users_all"]
    
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][0] = diff_in
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][1] = diff_el
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][2] = diff_all
        
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][3] = diff_in_pure
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][4] = diff_el_pure
        avg_travel_time_discount_minus_budget[toll_idx][budget_idx][5] = diff_all_pure

# avg_travel_time_discount_minus_budget

In [64]:
print("Users_in:")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 0]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 0]))
print()

print("Users_el:")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 1]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 1]))
print()

print("Users_all:")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 2]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 2]))
print()
print()

print("Users_in (pure):")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 3]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 3]))
print()

print("Users_el (pure):")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 4]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 4]))
print()

print("Users_all (pure):")
print("Max average time difference, discount minus budget, value:", \
          np.max(avg_travel_time_discount_minus_budget[:, :, 5]))
print("Min average time difference, discount minus budget, value:", \
          np.min(avg_travel_time_discount_minus_budget[:, :, 5]))

Users_in:
Max average time difference, discount minus budget, value: 0.6750000006149381
Min average time difference, discount minus budget, value: -0.37476475279833776

Users_el:
Max average time difference, discount minus budget, value: 0.1655177434105375
Min average time difference, discount minus budget, value: -0.6993750100286795

Users_all:
Max average time difference, discount minus budget, value: 2.7995834539495945e-08
Min average time difference, discount minus budget, value: -0.11710913788996757


Users_in (pure):
Max average time difference, discount minus budget, value: 0.6750000006149381
Min average time difference, discount minus budget, value: -0.37476475279833776

Users_el (pure):
Max average time difference, discount minus budget, value: 0.1655177434105375
Min average time difference, discount minus budget, value: -0.6993750100286795

Users_all (pure):
Max average time difference, discount minus budget, value: 2.7995834539495945e-08
Min average time difference, discount

In [55]:
avg_travel_time_discount_minus_budget[:, :, 2]

array([[ 2.79958345e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-9.37499875e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.56248653e-03, -2.49999780e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.0

In [57]:
edge_flows_d1 = solution_set_d1["edge_flows"]
users_el_d1 = solution_set_d1["users_el"]
users_in_d1 = solution_set_d1["users_in"]

edge_flows_bpure_1 = solution_set_bpure_1["edge_flows"]
users_el_bpure_1 = solution_set_bpure_1["users_el"]
users_in_bpure_1 = solution_set_bpure_1["users_in"]

print("edge_flows_d1:\n", edge_flows_d1)
print()
print("users_el_d1:\n", users_el_d1)
print()
print("users_in_d1:\n", users_in_d1)
print()

print()
print("edge_flows_bpure_1:\n", edge_flows_bpure_1)
print()
print("users_el_bpure_1:\n", users_el_bpure_1)
print()
print("users_in_bpure_1:\n", users_in_bpure_1)
print()

avg_travel_times_d1 = avg_travel_times_1(solution_set_d1, theta)
avg_travel_times_bpure_1 = avg_travel_times_1(solution_set_bpure_1, theta)

print()
print("avg_travel_times_d1:\n", avg_travel_times_d1)
print()
print("avg_travel_times_bpure_1:\n", avg_travel_times_bpure_1)
print()

NameError: name 'solution_set_bpure_1' is not defined

# From other files:

In [ ]:
# # Solver 2, Discount:

# def OptPL_discount(toll, discount_ratio, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible, 
#             vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
#               fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity, c_2 = bpr_true_capacity, 
#              a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope, b_2 = apx_slope):
#     """
#     Function to solve convex optimization problem given a particular toll and budget value
#     """
    
#     # Initialize model
#     m1 = gp.Model()
#     # Add variables to model
# #     users_in = m1.addVars(num_ineligible, 3, T, name = 'ineligible_val', lb = 0.0, ub = 2.0)
# #     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = 0.0, ub = num_ineligible * 2.0)
# #     users_el = m1.addVars(3, T, name = 'eligible_allocation', lb = 0.0, ub = 2.0)
#     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
    
# #     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
#     users_in = m1.addVars(num_ineligible, 2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
#     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
#     users_el = m1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
#     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = -10000.0, ub = 10000.0)   

#     ## Add constraints to Model 
    
#     # Total edge flow, express and general purpose lanes:
#     m1.addConstrs((edge_flows[j, t] == sum(users_in[i, j, t] for i in range(num_ineligible)) + \
#                   users_el[j, t] for j in range(2) for t in range(T)))
    
#     # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
#     m1.addConstrs((users_in[i, j, t]>=0 for i in range(num_ineligible) for j in range(2) for t in range(T)))
#     m1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
#     # Every eligible and ineligible user is assigned to one of three options: \
#     # (express paying, express with credit, non-express)
#     m1.addConstrs((sum(users_in[i, j, t] for j in range(2)) == 1 for i in range(num_ineligible) for t in range(T)))
#     m1.addConstrs((sum(users_el[j, t] for j in range(2)) == num_eligible for t in range(T)))
        
#     # Piecewise affine approximation:
#     m1.addConstrs((eps_flows[j, t] >= 0 for j in range(2) for t in range(T))) # Must be at least 0
#     m1.addConstrs((eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1 for t in range(T))) # Must be at least flow - capacity
#     m1.addConstrs((eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2 for t in range(T))) # Must be at least flow - capacity
    
    
#     ## To edit below:
#     # Set Objective
#     m1.setObjective(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2  
#                         + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 
#                         + sum( toll*users_in[i,0,t]/vot_ineligible[i,t] for i in range(num_ineligible))
#                         + sum( (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible[i,t] for i in range(num_eligible))
#         for t in range(T)), GRB.MINIMIZE)
    
#     m1.update()
    
#     return m1

# Scratch Work:

In [ ]:
# m_sample = gp.Model()
